## **Prerequisites**

### **Check what GPU you got**
If you didn't get the P100-PCIE GPU, click on the Runtime dropdown at the top of the page and Factory Reset Runtime 

In [0]:
!nvidia-smi

Fri May  1 00:38:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### **Mount and Install Packages**
Mount the data and set up Spacy


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.7MB 1.7MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=a56eb8da8223ce3f1349f6b96f08e2e27e05e3c09731c1f1a66f5ad405fe7e45
  Stored in directory: /tmp/pip-ephem-wheel-cache-2e6jnp4b/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfull

### **Important: Reset Runtime**

Note: there is a slight bug with Google Colab. After installing Spacy, you need to restart the Jupyter Notebook runtime.

There are two ways:
1. Click on the Runtime dropdown, and select "Restart Runtime". Once that is done, proceed to the next step (no need to remount the drive).
2. Run the code below. It will kill the current process, effectively restarting the runtime.

In [0]:
import os
os.kill(os.getpid(), 9)

##Get the data

### **Get a list of vocabs:**
The list of vocabs are already stored in the Google Drive folder; thus, we just have to load it.

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Translator/src/dataloader/')


def get_size_of_corpus(filepaths):
    """ Given a list of filepaths, it will return the total number of lines

        Parameters
        ----------
        filepaths : [ str ]
            A list of filepaths

        Returns
        -------
        num_lines : int
            The total number of lines in filepaths
    """

    def blocks(files, size=65536):
        while True:
            b = files.read(size)
            if not b:
                break
            yield b

    num_lines = 0
    for filepath in filepaths:
        with open(filepath, encoding="utf-8") as f:
            num_lines += sum(bl.count("\n") for bl in blocks(f))

    return num_lines


def get_tokens_from_line(line, spacy_instance):
    global PUNCTUATION_TABLE, LEGAL_CHARS_TABLE, LEGAL_CHARS

    # Make the text lowercase and remove the \n
    line = line.lower().strip()

    # Standardize single quotations
    line = (
        line.replace("\u2019", "'")
        .replace("\u0060", "'")
        .replace("\u00B4", "'")
        .replace("\u2018", "'")
        .replace("\u201A", "'")
        .replace("\u201B", "'")
        .replace("\u2039", "'")
        .replace("\u203A", "'")
        .replace("\u275B", "'")
        .replace("\u275C", "'")
        .replace("\u276E", "'")
        .replace("\u276F", "'")
    )

    # Standardize dashes
    line = (
        line.replace("\u00AD", "-")
        .replace("\u2013", "-")
        .replace("\u2014", "-")
        .replace("\u2015", "-")
        .replace("\u2212", "-")
        .replace("\u02D7", "-")
    )

    # Standardize double quotations
    line = (
        line.replace("\u201C", '"')
        .replace("\u201D", '"')
        .replace("\u2033", '"')
        .replace("\u00AB", '"')
        .replace("\u00BB", '"')
        .replace("\u201E", '"')
        .replace("\u201F", '"')
        .replace("\u275D", '"')
        .replace("\u275E", '"')
        .replace("\u301D", '"')
        .replace("\u301E", '"')
        .replace("\u301F", '"')
        .replace("\uFF02", '"')
    )

    # Standardize the ellipses
    line = line.replace("\u2026", "...")

    # Tokenize the text
    tokens = spacy_instance.tokenizer(line)

    # Remove tokens that are spaces, tabs, newlines, etc
    tokens = list(filter(lambda token: not token.is_space, tokens))

    # Extract only the text
    tokens = [token.text for token in tokens]

    # Replace tokens that contain numbers with 'NUM'
    num_vals = []
    new_tokens = []
    for token in tokens:
        new_token = token

        if any(char.isdigit() for char in token):
            new_token = "NUM"
            num_vals.append(token)

        new_tokens.append(new_token)
    
    tokens = new_tokens

    return tokens, num_vals


def read_transcription_files(filepaths, spacy_instance):
    """ Generate line info from data in a file for a given language

        Parameters
        ----------
        lang : {'en', 'fr'}
            Whether to tokenize the English sentences ('e') or French ('f').
        filenames : sequence
            Only tokenize sentences with matching names. If :obj:`None`, searches
            the whole directory in C-sorted order.

        Yields
        ------
        tokenized, filename, offs : list
            `tokenized` is a list of tokens for a line. `filename` is the source
            file. `offs` is the start of the sentence in the file, to seek to.
            Lines are yielded by iterating over lines in each file in the order
            presented in `filenames`.
    """
    for filepath in filepaths:
        with open(filepath, encoding="utf-8") as f:
            offs = f.tell()
            line = f.readline()
            while line:
                tokens, _ = get_tokens_from_line(line, spacy_instance)

                yield tokens, filepath, offs
                offs = f.tell()
                line = f.readline()


def get_parallel_text(dir_, langs):
    """ Get a list of all files in 'dir_' with a file extension that is in 'langs'

        Parameters
        ----------
        dir_ : str
            A path to the transcription dictionary
        langs : [str]
            A list of file extensions for the parallel texts

        Returns
        -------
        filenames : list
            A list of all parallel texts' file names without the file extension
    """
    # Get a set of files that has the lang file extension
    files = os.listdir(dir_)
    filenames = []
    for lang in langs:
        lang = "." + lang
        lang_filenames = set(
            filename[:-3] for filename in files if filename.endswith(lang)
        )
        filenames.append(lang_filenames)
    del files

    if len(filenames) == 0:
        raise ValueError(
            f"Directory {dir_} contains no transcriptions ending in {lang} "
        )

    # Get the set of files that has the same name
    transcriptions_filenames = filenames[0]
    for lang_filenames in filenames:
        transcriptions_filenames = transcriptions_filenames & lang_filenames

    transcriptions_filenames = sorted(transcriptions_filenames)

    if len(transcriptions_filenames) == 0:
        raise ValueError(
            f"Directory {dir_} contains no common files ending in {lang}."
            f"Are you sure this is the right directory?"
        )

    # Create the output
    parallel_text = []
    for filename in transcriptions_filenames:
        parallel_text.append(tuple([filename + "." + lang for lang in langs]))

    return parallel_text


def get_spacy_instance(lang, spacy_namespace=None):
    """ Returns the correct Spacy instance given the lang ISO.
        If the namespace is provided, it will use the namespace instead

        Parameters
        ----------
        lang : { 'en', 'fr' }
            The lang ISO
        spacy_namespace : str, optional
            The Spacy namespace

        Returns
        -------
        spacy_instance : Spacy
            The Spacy instance with the correct language
    """
    if spacy_namespace is None:
        if lang == "en":
            spacy_namespace = "en_core_web_sm"
        elif lang == "fr":
            spacy_namespace = "fr_core_news_sm"
        else:
            raise Exception("Unknown language: " + lang)

    spacy_instance = spacy.load(spacy_namespace, disable=["parser", "ner"])
    # spacy_instance.add_pipe(spacy_instance.create_pipe("sentencizer"))

    return spacy_instance

class VocabDataset:
    def __init__(self, word2id={}, id2word={}):
        """
            Initializes the Vocab dataset

            Parameters
            ----------
            word2id : { str : int }, optional
                A mapping of words to their ID

            id2word : { str : int }, optional
                A mapping of IDs to their words
        """

        self.word2id = word2id
        self.id2word = id2word

    def get_id2word(self):
        """ Returns a dictionary mapping an ID to a word

            Returns
            -------
            dictionary : { int : str }
                key is the ID of the word, and its value is the word itself
        """
        return self.id2word

    def get_word2id(self):
        """ Returns a dictionary mapping a word to an ID

            Returns
            -------
            dictionary : { str : int }
                key is the word itself, and its value is the ID of the word
        """
        return self.word2id


def load_vocabs_from_file(file_):
    """ Read self.word2id map from a file

        Parameters
        ----------
        file_ : str or file
            A file to read `word2id` from. If a path that ends with ``.gz``, it
            will be de-compressed via gzip.
    """
    if isinstance(file_, str):
        if file_.endswith(".gz"):
            with gzip.open(file_, mode="rt", encoding="utf8") as file_:
                return load_vocabs_from_file(file_)
        else:
            with open(file_, encoding="utf8") as file_:
                return load_vocabs_from_file(file_)

    word2id = dict()
    id2word = dict()

    for line in file_:
        line = line.strip()
        if not line:
            continue

        tokens = line.split()

        if len(tokens) == 2:
            word, id_ = tokens[0], tokens[1]
            id_ = int(id_)

            if id_ in id2word:
                raise ValueError(f"Duplicate id {id_}")
            if word in word2id:
                raise ValueError(f"Duplicate word {word}")

            word2id[word] = id_
            id2word[id_] = word

        else:
            raise Exception("Illegal vocab:", line)

    print("Loaded", len(word2id), "words")

    return VocabDataset(word2id, id2word)


def save_vocabs_to_file(vocabs, file_):
    """ Write vocabs.word2id map to a file

        Parameters
        ----------
        vocabs : VocabDataset
            The dataset to save
        file_ : str or file
            A file to write `word2id` to. If a path that ends with ``.gz``, it will be gzipped.
    """
    if isinstance(file_, str):
        if file_.endswith(".gz"):
            with gzip.open(file_, mode="wt", encoding="utf8") as file_:
                return save_vocabs_to_file(vocabs, file_)
        else:
            with open(file_, "w", encoding="utf8") as file_:
                return save_vocabs_to_file(vocabs, file_)

    id2word = vocabs.get_id2word()
    for i in range(len(id2word)):
        line = "{} {}\n".format(id2word[i], i)
        file_.write(line)


def build_vocabs_from_dir(
    train_dir_, lang, spacy_namespace=None, max_vocab=float("inf"), min_freq=0
):
  """ Build a vocabulary (words->ids) from transcriptions in a directory

      Parameters
      ----------
      train_dir_ : str
          A path to the transcription directory. ALWAYS use the training
          directory, not the test, directory, when building a vocabulary.

      lang : {'en', 'fr'}
          Whether to build the English vocabulary ('e') or the French one ('f').

      spacy_namespace : str
          It is the type of Spacy (ex: en_core_web_sm). if None, it will set
          it to 'en_core_web_sm' if lang is 'en'; else if 'fr' then 'fr_core_news_sm'

      max_vocab : int, optional
          The size of your vocabulary. Words with the greatest count will be
          retained.

      min_freq : int, optional
          The minimum frequency each word in the vocabulary needs to be
  """

  # Set up spacy for tokenisation
  spacy_instance = utils.get_spacy_instance(lang)

  # Get the language corpus
  transcriptions = utils.get_parallel_text(train_dir_, [lang])
  filepaths = [os.path.join(train_dir_, trans[0]) for trans in transcriptions]

  print("Building", lang, "vocab from", len(transcriptions), "transcriptions")

  # Build a counter of tokens
  word2count = Counter()

  corpus = utils.read_transcription_files(filepaths, spacy_instance)
  corpus_size = utils.get_size_of_corpus(filepaths)

  for tokenized, _, _ in tqdm(corpus, total=corpus_size):
      word2count.update(list(set(tokenized)))

  # Filter those that meet the frequency
  word2count = list(
      filter(lambda word_count: word_count[1] >= min_freq, word2count.items())
  )

  # Sort tokens in dec. frequency and cap it by max_vocab
  word2count = sorted(word2count, key=lambda kv: (kv[1], kv[0]), reverse=True)

  # Cap the number of words in the corpus
  if len(word2count) > max_vocab:
      word2count = word2count[0:max_vocab]

  word2id = dict((v[0], i) for i, v in enumerate(word2count))
  id2word = dict((i, v[0]) for i, v in enumerate(word2count))

  print("Built", len(word2id), "vocabs")

  return VocabDataset(word2id, id2word)

def combine_vocabs(vocab1, vocab2):
  combined_words = set()

  for key in vocab1.get_word2id():
    combined_words.add(key)

  for key in vocab2.get_word2id():
    combined_words.add(key)

  word2id = dict((word, index) for index, word in enumerate(combined_words))
  id2word = dict((index, word) for index, word in enumerate(combined_words))

  print('Built', len(word2id), 'vocabs')

  return VocabDataset(word2id, id2word)

models_dir = "/content/drive/My Drive/Translator/models/Hansard/"
french_vocabs = load_vocabs_from_file(models_dir + 'vocab.french.gz')
english_vocabs = load_vocabs_from_file(models_dir + 'vocab.english.gz')

combined_vocabs = combine_vocabs(english_vocabs, french_vocabs)
combined_vocabs = sorted([word for word in combined_vocabs.get_word2id()])


Loaded 33639 words
Loaded 25370 words
Built 52275 vocabs


### **Tokenize and split the dataset**
We will have three types of datasets:
1. Training data: it is the data used to train our model
2. Validation (val) data: it is the data used to test our model at each step of the training process
3. Test data: it is the data used to test our model after all the training is done

How to get the three types of data?
* The test set is already in the `Test` folder
* The validation set is a piece of the data in the `Train`` folder

The code to get our three types of datasets is:

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Translator/src/dataloader')

import os
from tqdm.notebook import tqdm

import torch

class Seq2VecDataset(torch.utils.data.Dataset):
  def __init__(self, dir_: str, vocabs: list, langs: list):
    ''' Reads in the sentences in `dir_`, convert each word into its numerical token, 
        and tags it with its language

        Parameters
        ----------
        dir_ : str
            The directory with the training data
        vocabs : [ str ]
            A list of vocabs
        lang : [ str ]
            The set of languages to capture in `dir_`
    '''

    pairs = []
    word2index = { word: index for index, word in enumerate(vocabs)}
    
    for lang_index, lang in enumerate(langs):

      # Get the spacy instance
      spacy_instance = get_spacy_instance(lang)

      # Get all the filenames with that language
      transcriptions = get_parallel_text(dir_, [lang])
      
      # Get all the filepaths with that language
      filepaths = [os.path.join(dir_, trans[0]) for trans in transcriptions]

      # Get the iterator that will read and tokenize all the content in filepaths
      iterator = read_transcription_files(filepaths, spacy_instance)

      # Get the number of sentences in entire corpus with that language
      corpus_size = get_size_of_corpus(filepaths)

      for (f, f_fn, _), in tqdm(iterable=zip(iterator), total=corpus_size):
        if not f:
          continue

        # Ignore sentences with no words in vocabs
        has_known_word = sum([1 if word in word2index else 0 for word in f]) > 0
        if not has_known_word:
          continue

        pairs.append((f, lang_index))
    
    self.langs = langs
    self.vocabs = vocabs
    self.pairs = pairs
    self.word2index = word2index
  
  def __len__(self):
    ''' Returns the number of sentences in this dataset '''
    return len(self.pairs)

  def __getitem__(self, i):
    ''' Returns the i-th sentence in this dataset '''
    f, lang_index = self.pairs[i]

    # Get the bag of words for f where vec[i] = 1 if word i exists; else 0
    F = torch.zeros(len(self.vocabs))
    for word in f:
      if word in self.word2index:
        index = self.word2index[word]
        F[index] = 1

    Y = torch.tensor(lang_index)

    return F, Y

utils.get_spacy_instance('fr')
train_dir = "/content/drive/My Drive/Translator/data/Hansard/Training"
test_dir = "/content/drive/My Drive/Translator/data/Hansard/Testing"

num_epochs = 10
batch_size = 32
device = torch.device('cuda')
train_test_split_ratio = 0.75

dataset = Seq2VecDataset(train_dir, combined_vocabs, ['en', 'fr'])

num_training_data = int(len(dataset) * train_test_split_ratio)
num_val_data = len(dataset) - num_training_data

train_dataset, val_dataset = torch.utils.data.random_split(
  dataset, [num_training_data, num_val_data]
)

train_dataloader = torch.utils.data.DataLoader(
  train_dataset, 
  batch_size=batch_size, 
  shuffle=True,
  pin_memory=(device.type == 'cuda'),
  num_workers=4
)
val_dataloader = torch.utils.data.DataLoader(
  val_dataset, 
  batch_size=batch_size, 
  shuffle=True,
  pin_memory=(device.type == 'cuda'),
  num_workers=4
)

test_dataset = Seq2VecDataset(test_dir, combined_vocabs, ['en', 'fr'])
test_dataloader = torch.utils.data.DataLoader(
  test_dataset, 
  batch_size=batch_size, 
  shuffle=True,
  pin_memory=(device.type == 'cuda'),
  num_workers=4
)

## Build the Classifier

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

class Seq2VecNN(nn.Module):
  def __init__(self, vocab_size, num_classes, num_neurons_per_layer=[1000, 1000, 1000]):

    super().__init__()

    self.vocab_size = vocab_size
    self.num_classes = num_classes

    layers = []

    prev_layer_count = vocab_size
    for num_neurons in num_neurons_per_layer:
      layers.append(nn.Linear(prev_layer_count, num_neurons))
      layers.append(nn.ReLU())
      prev_layer_count = num_neurons

    self.feedforward_layer = nn.Sequential(*layers)

    self.output_layer = nn.Linear(prev_layer_count, self.num_classes)
      
  def forward(self, F):
    ''' Given a batch of sequences, and its sequence lengths, output a softmax of
        its class

        Parameters
        ----------
        F : torch.LongTensor (N, self.vocab_size)
            It is a batch of bag-of-words

        Returns
        -------
        logits_t : torch.FloatTensor (N, self.vocab_size)
            It is a un-normalized distribution over the classes for the n-th sequence:
            Pr_b(i) = softmax(logits_t[i]) for i in self.num_classes
    '''
    x = self.feedforward_layer(F)
    return self.output_layer(x)
        

## **Training:**
We are going to train our model and see if for each epoch it improves its predictions on the validation set

How we train it:
* It uses teacher forcing:
  1. First, we have the source and target sentences
  2. Then, we feed the source sentence into the Encoder. The encoder returns the attended source sentence.
  3. Next, we feed the attended source sentence and the target sentence into the decoder
  4. We check if the output of the decoder is the same as the target sentence

How we make predictions:
* It is similar to RNNs, where we feed in the source input to the encoder, feed in an SOS in the decoder, and we take the outputs of the previous decoder as inputs to the next decoder
* In more detail:
  1. First, we have the source sentence
  2. Then, we feed the source sentence into the Encoder to get an attended version of the source sentence
  3. Next, we feed an SOS token and the attended source sentence in the Decoder as the first input to our decoder
  4. We get the outputs of the decoder and use it as the next token to feed as the second input to our decoder
  5. Repeat 3-4 until we get an EOS token

### **How to train our model for one epoch:**

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

def train_for_one_epoch(model, loss_function, optimizer, train_dataloader, device):
  model.train()

  train_loss = 0.0
  train_accuracy = 0.0

  for F, Y in tqdm(train_dataloader, total=len(train_dataloader)):

    # Send data to device
    F = F.to(device)
    Y = Y.to(device)   

    # Forward-prop with the model
    optimizer.zero_grad()
    logits = model(F)
    
    # Compute the loss
    batch_loss = loss_function(logits, Y)
    train_loss += batch_loss.item()

    # Compute the accuracy
    _, predictions = torch.max(torch.round(torch.sigmoid(logits)), 1)    
    batch_accuracy = predictions.eq(Y).sum().float().item() / Y.shape[0]
    train_accuracy += batch_accuracy
    
    batch_loss.backward()
    optimizer.step()

    del F, Y

  train_loss /= len(train_dataloader)
  train_accuracy /= len(train_dataloader)

  return train_loss, train_accuracy


### **How to evaluate our model:**

In [0]:
def evaluate_model(model, loss_function, test_dataloader, device):

  model.eval()

  eval_loss = 0
  eval_accuracy = 0
  num_sequences = 0

  for F, Y in tqdm(test_dataloader, total=len(test_dataloader)):

    # Send data to device
    F = F.to(device)
    Y = Y.to(device)

    # Get predictions
    logits = model(F)

    # Compute the loss
    eval_loss = loss_function(logits, Y).item()

    # Compute the accuracy
    _, predictions = torch.max(torch.round(torch.sigmoid(logits)), 1)    
    batch_accuracy = predictions.eq(Y).sum().float().item() / Y.shape[0]
    eval_accuracy += batch_accuracy

    del F, Y

  eval_loss /= len(test_dataloader)
  eval_accuracy /= len(test_dataloader)

  return eval_loss, eval_accuracy
  


### **How to train our model for many epochs:**

In [0]:
import torch
from tqdm.notebook import tqdm

def train():
    global combined_vocabs, train_dataloader, val_dataloader

    device = torch.device("cuda")

    model = Seq2VecNN(len(combined_vocabs), 2, num_neurons_per_layer=[100, 25])
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters())

    patience = 3 #float("inf")
    num_epochs = float("inf")
    
    best_eval_loss = float("inf")

    num_poor = 0
    epoch = 1
    
    while epoch <= num_epochs and num_poor < patience:
        
      loss_function = torch.nn.CrossEntropyLoss()

      train_loss, train_accuracy = train_for_one_epoch(model, loss_function, optimizer, train_dataloader, device)

      eval_loss, eval_accuracy = evaluate_model(model, loss_function, val_dataloader, device)

      print(f'Epoch={epoch} Train-Loss={train_loss} Train-Acc={train_accuracy} Test-Loss={eval_loss} Test-Acc={eval_accuracy} Num-Poor={num_poor}')

      if eval_loss >= best_eval_loss:
        num_poor += 1

      else:
        num_poor = 0
        best_eval_loss = eval_loss

      epoch += 1

    return model

trained_model = train()


    



Epoch=1 Train-Loss=0.012065429525193669 Train-Acc=0.9932020680223819 Test-Loss=5.233084163942926e-06 Test-Acc=0.9932454121257672 Num-Poor=0



Epoch=2 Train-Loss=0.010305593614350598 Train-Acc=0.9935685304270364 Test-Loss=1.0664742213658607e-05 Test-Acc=0.9932003945885005 Num-Poor=0



Epoch=3 Train-Loss=0.010036695400318535 Train-Acc=0.9910672342158009 Test-Loss=3.3599010583647476e-11 Test-Acc=0.9934137385694601 Num-Poor=1



Epoch=4 Train-Loss=0.009923419686731938 Train-Acc=0.9916603161015534 Test-Loss=2.0159406350188486e-10 Test-Acc=0.9931964800200426 Num-Poor=0



Epoch=5 Train-Loss=0.009935518112526723 Train-Acc=0.98790935046768 Test-Loss=5.674499565238241e-10 Test-Acc=0.9931925654515846 Num-Poor=1


## **Testing**
We are going to test our model on the test set

In [0]:
def test():
  ''' Used to test the model on the testing data'''
  global test_dataloader
  global trained_model

  device = torch.device("cuda")  

  # Evaluate the model
  loss_function = torch.nn.CrossEntropyLoss()
  test_loss, test_accuracy = evaluate_model(trained_model, loss_function, test_dataloader, device)

  print(f"Test loss={test_loss}, Test Accuracy={test_accuracy}")

test()


Test loss=2.8081064123368606e-06, Test Accuracy=0.9941644272232986
